In [2]:
#   add email, password
import sys
import os
import csv
import json
import pandas as pd
import time
import random

def str_time_prop(start, end, time_format, prop):
    stime = time.mktime(time.strptime(start, time_format))
    etime = time.mktime(time.strptime(end, time_format))
    ptime = stime + prop * (etime - stime)
    return time.strftime(time_format, time.localtime(ptime))

def random_date(start, end, prop):
    return str_time_prop(start, end, '%Y/%m/%d %I:%M %p', prop)



list_anime_revs = [ 
                'username' , 
                'anime_id',
                'my_watched_episodes' , 
                'my_score' , 
                ]


#   DONE :
expanded_list_anime_revs = [ 
                    'user_id'           #  get
                ,   'username'      # have
                ,   'anime_id'      # have
                ,   'animename'         #  get
                ,   'my_score'      # have
                ,   'my_rev_time'       #  rng
                ,   'my_rev_text'    
            ]


list_for_graph = [
        'user_id' 
    ,   'anime_id'
    ,   'my_status'
    ,   'my_watched_episodes'
]


print(f'length of target cols : {len(list_anime_revs)}')



do_merge_reviews = True
if do_merge_reviews == True :

    t0 = time.time()

    # read from source
    src = './000_unpolished_csv/animelists_cleaned.csv'
    file_base = pd.read_csv(src)
    load_time = time.time()

    extracted = file_base[list_anime_revs]
    print(f'length of extr cols : {extracted.columns.values.size} ' )
    print(f'{extracted.iloc[0]}\n\n')
    t1 = time.time()
    print(f'time to load : {t1-t0}')



    #   ADD REVIEW TIME
    extracted['my_rev_time'] = random_date("2024/05/01 1:30 AM", "2024/06/10 4:50 PM", random.random())
    #   extracted['my_rev_text'] = ''
    print(f'{extracted.iloc[0]} \n\n')
    t2 = time.time()
    print(f'time to rng timestamp : {t2-t1}')



    #   ADD USER_ID based on USERNAME
    usernamefile = './final_csv/userid_username_graph.csv'
    with open(usernamefile, 'rb') as namefile :
        namedf = pd.read_csv(namefile)
    extracted = pd.merge(
                    extracted
                ,   namedf
                ,   on= 'username'
                ,   how = 'left'
    )
    print(f'{extracted.iloc[0]} \n\n')
    t3 = time.time()
    print(f'time to merge username : {t3-t2}')



    #   ADD ANIMENAME based on ANIME_ID
    animefilename = './final_csv/animeid_aniname_graph.csv'
    with open(animefilename, 'rb') as anifile :
        animedf = pd.read_csv(anifile)
    animedf.rename(columns={ 'anime_id':'anime_id' , 'title':'animename' })
    extracted = pd.merge(
                    extracted
                ,   animedf
                ,   on= 'anime_id'
                ,   how = 'left'
    )
    print(f'{extracted.iloc[0]} \n\n')
    t4 = time.time()
    print(f'time to merge anime_id : {t4-t3}')



    #   ADD REVIEW TEXT 
    import pickle
    reviewfilename = './final_csv/ciaociaorevs.csv'
    with open(reviewfilename, 'rb') as redfil :
        loaded = pd.read_csv(redfil)
        #   redfil.close()
    #   revindex_vals = ['username', 'anime_id', 'my_score', 'text']
    #   revdfff = pd.DataFrame(loaded, columns=revindex_vals)

    merged = extracted.merge(loaded, how='left', on=['username', 'anime_id', 'my_score'] )

    print(f'{merged.iloc[0]} \n\n')
    expand_time = time.time()  
    t5 = time.time()
    print(f'time to merge reviews : {t5-t4}')




    # SAVE to destination

    dst = './final_csv/zzz_anime_revs.csv'
    if not os.path.exists(dst):
        with open(dst, 'w'): pass 
    merged.to_csv(dst, index=False)


    jdst = './final_csv/zzz_anime_revs.csv'
    if not os.path.exists(jdst):
        with open(jdst, 'w'): pass 
    merged.to_json(jdst, index=False)



    save_time = time.time()
    t6 = time.time()
    print(f'time to save : {t6-t5}')
    #   print(extracted.iloc[0])
    print(f'total time : {t6-t0} seconds')

    print(f'\n\n{merged.iloc[0]} \n\n')

    print("DBG -> the end")

length of target cols : 4


FileNotFoundError: [Errno 2] No such file or directory: './000_unpolished_csv/animelists_cleaned.csv'